In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd
import os
import logging

class YelpSpider(scrapy.Spider):
    name = 'yelp_spider'
    start_urls = ['https://www.yelp.fr/']

    def parse(self, response):
        # Fill the search form with default parameters: 'restaurant japonais' as search keywords, 'Paris' as search location
        search_keywords = "restaurant japonais"
        search_location = "Paris"
        
        # Extract search form input fields
        formdata = {
            'find_desc': search_keywords,  # Search for restaurants
            'find_loc': search_location,  # Search in Paris
        }
        
        # Submit the form
        yield scrapy.FormRequest.from_response(
            response,
            formdata=formdata,
            callback=self.after_search
        )

    def after_search(self, response):
        # Parse the first page results
        url = 'https://www.yelp.fr/'

        for result in response.css('div.y-css-snigah'):
            yield {
                'name': result.css('a.y-css-1x1e1r2::text').get(),
                'url': result.css('a.y-css-1x1e1r2::attr(href)').get(),
            }

        # Follow pagination to the next pages
        next_page = response.css('a.next-link::attr(href)').get()
        if next_page:
            yield response.follow(next_page, callback=self.after_search)



filename = "restaurants.json"

if filename in os.listdir("src/"):
    os.remove("src/" + filename)

# To run the spider and store the results in a file
process = CrawlerProcess(settings={
    'USER_AGENT': 'Mozilla/4.0',
    'LOG_LEVEL':logging.INFO,
    "FEEDS":{
        "src/" + filename : {"format":"json"}
    }
}
    
)

process.crawl(YelpSpider)
process.start()


2025-02-24 17:19:46 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2025-02-24 17:19:46 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.0.15 3 Sep 2024), cryptography 43.0.0, Platform Windows-11-10.0.26100-SP0
2025-02-24 17:19:46 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-24 17:19:46 [py.warnings] WARNING: c:\Users\darla\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu

In [2]:
pd.read_json('src/' + filename)

,name,url
0,Sanukiya,/biz/sanukiya-paris?osq=restaurant+japonais
1,Sushi Yaki,/biz/sushi-yaki-paris-4?osq=restaurant+japonais
2,Okuda,/biz/okuda-paris?osq=restaurant+japonais
3,Aki,/biz/aki-paris-2?osq=restaurant+japonais
4,Kodawari Ramen,/biz/kodawari-ramen-paris-9?osq=restaurant+jap...
...,...,...
235,Itadaki,/biz/itadaki-paris?osq=restaurant+japonais
236,Wahaha,/biz/wahaha-malakoff?osq=restaurant+japonais
237,Miam,/biz/miam-paris-3?osq=restaurant+japonais
238,Isami,/biz/isami-paris?osq=restaurant+japonais
